## Setup

First save the data you want to inspect to your google drive, saving it under a folder 'Review1', should you decide to use a different folder name it will need to be changed in the path variable.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/Review1'
import numpy as np
Images = np.load(path + '/Images.npy')
Labels = np.load(path + '/Labels.npy').astype(int)
num_Images = np.shape(Images)[0]



In [ ]:
np.shape(Images)


In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [ ]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [ ]:

# Names of the integer classes, i.e., 0 -> #, 1 -> $, etc.
class_names = ['#', '$', '%', '&', 'Ξ','Π','Σ','Θ','Γ','⊙','◯','≤','<','>','≥','≪','≫','∈','⊂','⊄','∉','≅','≈','≠','=']

## Visualizing a single image

Now we're going to simply log the first training image in your training set in TensorBoard.

Before you do that, examine the shape of your training data:

In [ ]:
class_names = ['#', '$', '%', '&', 'Ξ','Π','Σ','Θ','Γ','⊙','◯','≤','<','>','≥','≪','≫','∈','⊂','⊄','∉','≅','≈','≠','=']
print("Shape: ", Images[0].shape)
print("Label: ", Labels[0], "->", class_names[Labels[0]])

Notice that the shape of each image in the data set is a rank-2 tensor of shape (150, 150), representing the height and the width.

However, ```tf.summary.image()``` expects a rank-4 tensor containing ```(batch_size, height, width, channels)```. Therefore, the tensors need to be reshaped. 

You're logging only one image, so ```batch_size``` is 1. The images are grayscale, so set ```channels``` to 1.

In [ ]:
# Reshape the image for the Summary API.
img = np.reshape(Images[200], (-1, 150, 150, 1))

You're now ready to log this image and view it in TensorBoard.

In [ ]:
# Clear out any prior log data.
!rm -rf logs

# Sets up a timestamped log directory.
logdir = "logs/train_data/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)

# Using the file writer, log the reshaped image.
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)

Now, use TensorBoard to examine the image. Wait a few seconds for the UI to spin up.

In [ ]:
%tensorboard --logdir logs/train_data

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_single.png?raw=1"/> -->

The "Images" tab displays the image you just logged.

The image is scaled to a default size for easier viewing. If you want to view the unscaled original image, check "Show actual image size" at the upper left. 

Play with the brightness and contrast sliders to see how they affect the image pixels.

## Visualizing multiple images

Logging one tensor is great, but what if you wanted to log multiple training examples?

Simply specify the number of images you want to log when passing data to ```tf.summary.image()```.

In [ ]:
with file_writer.as_default():
  # Don't forget to reshape.
  images = np.reshape(Images[0:25], (-1, 150, 150, 1))
  tf.summary.image("25 training data examples", images, max_outputs=25, step=0)

%tensorboard --logdir logs/train_data

## Logging arbitrary image data

What if you want to visualize an image that's not a tensor, such as an image generated by [matplotlib](https://matplotlib.org/)?

You need some boilerplate code to convert the plot to a tensor, but after that, you're good to go.

In the code below, you'll log all of the images in a nice 25 x 25 grid using matplotlib's ```subplot()``` function. You'll then view the grid in TensorBoard:

In [ ]:
# Clear out prior logging data.
!rm -rf logs/plots

logdir = "logs/plots/" + datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=1)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid(start_pos = 0):
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[Labels[i + start_pos]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Images[i + start_pos], cmap=plt.cm.gray)
  
  return figure

for i in range(num_Images//25):
  # Prepare the plot
  figure = image_grid(i*25)
  # Convert to image and log
  with file_writer.as_default():
    tf.summary.image("Training data_" + str(i), plot_to_image(figure), step=0)


%tensorboard --logdir logs/plots
